In [67]:
import tensorflow
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
dados = pd.read_csv('../dados/dados_tratado.csv')

In [3]:
dados.diagnosis = dados.diagnosis.map(lambda x: 0 if x == 1 else 1)

In [48]:
dados.head()

,age,sex,height,weight,qrs_duration,p_r_interval,q_t_interval,t_interval,p_interval,qrs,T,P,QRST,heart_rate,q_wave,r_wave,s_wave,R_wave,S_wave,diagnosis
0,75,0,190,80,91,193,371,174,121,-16,13,64,-2,63,0,52,44,0,0,1
1,56,1,165,64,81,174,401,149,39,25,37,-17,31,53,0,48,0,0,0,1
2,54,0,172,95,138,163,386,185,102,96,34,70,66,75,0,40,80,0,0,1
3,55,0,175,94,100,202,380,179,143,28,11,-5,20,71,0,72,20,0,0,0
4,13,0,169,51,100,167,321,174,91,107,66,52,88,84,0,36,48,0,0,1


In [201]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   age           420 non-null    int64
 1   sex           420 non-null    int64
 2   height        420 non-null    int64
 3   weight        420 non-null    int64
 4   qrs_duration  420 non-null    int64
 5   p_r_interval  420 non-null    int64
 6   q_t_interval  420 non-null    int64
 7   t_interval    420 non-null    int64
 8   p_interval    420 non-null    int64
 9   qrs           420 non-null    int64
 10  T             420 non-null    int64
 11  P             420 non-null    int64
 12  QRST          420 non-null    int64
 13  heart_rate    420 non-null    int64
 14  q_wave        420 non-null    int64
 15  r_wave        420 non-null    int64
 16  s_wave        420 non-null    int64
 17  R_wave        420 non-null    int64
 18  S_wave        420 non-null    int64
 19  diagnosis     420 non-null   

In [5]:
x = dados[['age', 'sex', 'height', 'weight', 'qrs_duration', 'p_r_interval', 'q_t_interval', 't_interval', 'p_interval',
          'qrs', 'T', 'P', 'QRST', 'heart_rate', 'q_wave', 'r_wave', 's_wave', 'R_wave', 'S_wave']]
y = dados['diagnosis']

In [6]:
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size= 0.25, stratify=y)
scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

In [7]:
treino_x

array([[-1.17519681,  0.84930058, -0.28005755, ...,  0.7352736 ,
        -0.05643326,  0.        ],
       [-0.0683254 ,  0.84930058, -0.19737889, ..., -0.98319355,
        -0.05643326,  0.        ],
       [ 0.23913889, -1.17743944,  0.18845485, ...,  1.11715519,
        -0.05643326,  0.        ],
       ...,
       [-1.29818252,  0.84930058, -0.14225979, ..., -0.98319355,
        -0.05643326,  0.        ],
       [-0.37578968, -1.17743944, -0.14225979, ..., -0.98319355,
        -0.05643326,  0.        ],
       [ 0.48511031,  0.84930058, -0.14225979, ..., -0.98319355,
        -0.05643326,  0.        ]])

In [8]:
tensor_treino_x = tensorflow.convert_to_tensor(treino_x)
tensor_teste_x = tensorflow.convert_to_tensor(teste_x)
tensor_treino_y = tensorflow.convert_to_tensor(treino_y)
tensor_teste_y = tensorflow.convert_to_tensor(teste_y)

In [9]:
tensor_treino_x

<tf.Tensor: shape=(315, 19), dtype=float64, numpy=
array([[-1.17519681,  0.84930058, -0.28005755, ...,  0.7352736 ,
        -0.05643326,  0.        ],
       [-0.0683254 ,  0.84930058, -0.19737889, ..., -0.98319355,
        -0.05643326,  0.        ],
       [ 0.23913889, -1.17743944,  0.18845485, ...,  1.11715519,
        -0.05643326,  0.        ],
       ...,
       [-1.29818252,  0.84930058, -0.14225979, ..., -0.98319355,
        -0.05643326,  0.        ],
       [-0.37578968, -1.17743944, -0.14225979, ..., -0.98319355,
        -0.05643326,  0.        ],
       [ 0.48511031,  0.84930058, -0.14225979, ..., -0.98319355,
        -0.05643326,  0.        ]])>

In [10]:
normalizer = tensorflow.keras.layers.Normalization(axis=-1)
normalizer.adapt(tensor_treino_x)

In [175]:
# camadas
model = keras.Sequential([
    normalizer,
    keras.layers.Dense(340, activation=tensorflow.nn.relu),
    keras.layers.Dense(170, activation=tensorflow.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(170, activation=tensorflow.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(2, activation=tensorflow.nn.softmax)
                         ])

In [176]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [199]:
model.fit(tensor_treino_x, tensor_treino_y, epochs=10, validation_split=0.2)

Epoch 1/10
8/8 [==============================] - 0s 8ms/step - loss: 5.6505e-04 - accuracy: 1.0000 - val_loss: 4.6456 - val_accuracy: 0.6825
Epoch 2/10
8/8 [==============================] - 0s 5ms/step - loss: 8.8743e-04 - accuracy: 1.0000 - val_loss: 4.6631 - val_accuracy: 0.6825
Epoch 3/10
8/8 [==============================] - 0s 5ms/step - loss: 5.1266e-04 - accuracy: 1.0000 - val_loss: 4.6865 - val_accuracy: 0.6825
Epoch 4/10
8/8 [==============================] - 0s 4ms/step - loss: 7.4486e-04 - accuracy: 1.0000 - val_loss: 4.7456 - val_accuracy: 0.6667
Epoch 5/10
8/8 [==============================] - 0s 3ms/step - loss: 4.3946e-04 - accuracy: 1.0000 - val_loss: 4.7925 - val_accuracy: 0.6667
Epoch 6/10
8/8 [==============================] - 0s 5ms/step - loss: 2.5327e-04 - accuracy: 1.0000 - val_loss: 4.8254 - val_accuracy: 0.6667
Epoch 7/10
8/8 [==============================] - 0s 5ms/step - loss: 3.4846e-04 - accuracy: 1.0000 - val_loss: 4.8497 - val_accuracy: 0.6667
Epoch 

In [69]:
probabilidades = model.predict(tensor_teste_x)

4/4 [==============================] - 0s 1ms/step


In [70]:
previsoes = [np.argmax(probabilidade) for probabilidade in probabilidades]

In [74]:
pd.options.display.max_rows = None
resultado_frame = pd.DataFrame({'Previsão': previsoes, 'Real': tensor_teste_y})

In [200]:
loss_test, accuracy_test = model.evaluate(tensor_teste_x, tensor_teste_y)
print(f'perda teste: {loss_test}')
print(f'Acurácia teste: {accuracy_test}')

4/4 [==============================] - 0s 2ms/step - loss: 1.8951 - accuracy: 0.7429
perda teste: 1.8951495885849
Acurácia teste: 0.7428571581840515
